# Build Your First AI Agent with LangGraph

Welcome to this hands-on workshop!

## Context  

Large Language Models (LLMs) are powerful, but on their own they can feel limited, often just responding to prompts without much structure. Agents, on the other hand, allow LLMs to **reason, decide, and act** using tools, making them much more practical for real-world tasks.  

The challenge is that building agents isn’t always straightforward. You often need more **control and precision**: maybe your agent should always check the web before answering certain questions, or behave differently depending on the situation.  

This is where [LangGraph](https://langchain-ai.github.io/langgraph/) comes in. LangGraph is a framework for building **agents and multi-agent applications**, giving you more control over workflows, making them reliable and production-ready.  

In this workshop, we’ll be building a simple **ReACT Agent** that:  
- Uses an LLM for general conversation and creative tasks  
- Calls a search tool when it needs real-time information  
- Decides the right path (LLM or search) based on your question  

By the end, you’ll have a working agent that can **chat and fetch real-world answers**.  

## Workshop Structure  

This workshop is organized into a set of Jupyter notebooks. Each notebook builds on the previous one:  
1. **Setup** – Installing dependencies, Load environment variables, and test API keys
2. **Intro to LangGraph** – What is LangGraph and it's key concepts
3. **Building Blocks** – Define nodes, add edges and state
4. **Simple ReAct Agent** – Use an LLM as a node, add a tool, and wrap into an agent graph
5. **Research Agent** – Running real examples  

You can run the notebooks directly, follow along with the code, or explore them afterward at your own pace.  

## Setup  

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! By default, the course will use [ChatGroq](https://python.langchain.com/docs/integrations/chat/groq/) and [ChatGoogleGenerativeAI](https://python.langchain.com/docs/integrations/chat/google_generative_ai/) as they provide free APIs for LLMs. As noted, please ensure that you have an `GROQ_API_KEY` and `GOOGLE_API_KEY`.

Let's check that your `OPENAI_API_KEY` and `GOOGLE_API_KEY` is set and, if not, you will be asked to enter it.

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_groq langchain-google-genai langchain_core langchain_community langchain-tavily

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [4]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")

If you do not have your Groq API key yet, you may obtain one [here](https://console.groq.com/keys).

In [5]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GOOGLE_API_KEY")

If you do not have your Gemini API key yet, you may obtain one [here](https://aistudio.google.com/app/apikey).

If your school or organization restricts access to Google Gemini Studio, try using a personal Google account to generate your API key.

[Here](https://python.langchain.com/v0.2/docs/how_to/#chat-models) is a useful how-to for everything you can do with chat models—we’ll highlight a few key points below.  

If you’ve run `pip install -r requirements.txt` as noted in the README, you’ve installed the `langchain-groq` and `langchain-google-genai` packages. With these, you can instantiate both `ChatGroq` and `ChatGoogleGenerativeAI` model objects.  

- You can see the **rate limits** for Groq models [here](https://console.groq.com/docs/rate-limits) and Google Gemini models [here](https://ai.google.dev/gemini-api/docs/rate-limits).  
- By default, the notebooks use **`llama-3.3-70b-versatile`**, which is optimized for a wide range of natural language processing tasks, delivers strong benchmark performance, and maintains efficiency across diverse applications ([see more here](https://console.groq.com/docs/model/llama-3.3-70b-versatile)).  

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) you can set with chat models. Two of the most common are:  

* `model`: the name of the model  
* `temperature`: the sampling temperature  

The `temperature` parameter controls the randomness or creativity of the model’s output:  

- **Low temperature (close to 0)** → deterministic, focused, and factual (best for accuracy-critical tasks)  
- **High temperature (close to 1)** → more diverse, creative, and exploratory (best for brainstorming or open-ended tasks)  


In [6]:
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI

llama_chat = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)
gemini_chat = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: provides the "answer in progress," allowing for real-time interaction and feedback.
* `invoke`: provides the "final answer" all at once.

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [7]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello World", name="Naza")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
llama_chat.invoke(messages)

AIMessage(content="Hello World. It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 37, 'total_tokens': 63, 'completion_time': 0.06605359, 'prompt_time': 0.010286471, 'queue_time': 0.045461209, 'total_time': 0.076340061}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f2cb492b-55f1-43ba-9bfa-c0d5981e1000-0', usage_metadata={'input_tokens': 37, 'output_tokens': 26, 'total_tokens': 63})

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.

In [8]:
gemini_chat.invoke("Hello World")

AIMessage(content="Hello World!\n\nIt's great to hear from you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--b9f64c51-fe99-4b49-9bae-26ce06d07e7b-0', usage_metadata={'input_tokens': 3, 'output_tokens': 52, 'total_tokens': 55, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 32}})

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.

## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. We will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [9]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")
        
_set_env("TAVILY_API_KEY")

In [10]:
from langchain_tavily import TavilySearch

tavily_search = TavilySearch(max_results=3)
search_docs = tavily_search.invoke("What is LangGraph?")

In [11]:
search_docs

{'query': 'What is LangGraph?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.ibm.com/think/topics/langgraph',
   'title': 'What is LangGraph?',
   'content': 'LangGraph, created by LangChain, is an open source AI agent framework designed to build, deploy and manage complex generative AI agent workflows. At its core, LangGraph uses the power of graph-based architectures to model and manage the intricate relationships between various components of an AI agent workflow. LangGraph illuminates the processes within an AI workflow, allowing full transparency of the agent’s state. By combining these technologies with a set of APIs and tools, LangGraph provides users with a versatile platform for developing AI solutions and workflows including chatbots, state graphs and other agent-based systems. **Nodes**: In LangGraph, nodes represent individual components or agents within an AI workflow. LangGraph uses enhanced decision-making by modeling co